In [56]:
import numpy as np;
import pandas as pd
from scipy.spatial import distance_matrix

In [57]:
dataset = pd.read_csv('heirarchical.csv')
labels=np.asarray(dataset.iloc[:,0]).tolist()
datapoints = dataset.iloc[:,1:]

In [58]:
dist_mat = distance_matrix(datapoints,datapoints,p=2)

In [59]:
dissimilarity_matrix = pd.DataFrame(dist_mat,index=labels, columns=labels)
dissimilarity_matrix

,A,B,C,D,E
A,0.000000,4.000000,1.000000,3.000000,3.162278
B,4.000000,0.000000,4.123106,5.000000,1.414214
C,1.000000,4.123106,0.000000,2.000000,3.000000
D,3.000000,5.000000,2.000000,0.000000,3.605551
E,3.162278,1.414214,3.000000,3.605551,0.000000


In [60]:
def avg_dissim_within_group_element(ele, element_list):
    max_diameter = -np.inf
    sum_dissm = 0
    for i in element_list:
        sum_dissm += dissimilarity_matrix[ele][i]   
        if( dissimilarity_matrix[ele][i]  > max_diameter):
            max_diameter = dissimilarity_matrix[ele][i]
    if(len(element_list)>1):
        avg = sum_dissm/(len(element_list)-1)
    else: 
        avg = 0
    return avg

def avg_dissim_across_group_element(ele, main_list, splinter_list):
    if len(splinter_list) == 0:
        return 0
    sum_dissm = 0
    for j in splinter_list:
        sum_dissm = sum_dissm + dissimilarity_matrix[ele][j]
    avg = sum_dissm/(len(splinter_list))
    return avg
    
    
def splinter(main_list, splinter_group):
    most_dissm_object_value = -np.inf
    most_dissm_object_index = None
    for ele in main_list:
        x = avg_dissim_within_group_element(ele, main_list)
        y = avg_dissim_across_group_element(ele, main_list, splinter_group)
        diff= x -y
        if diff > most_dissm_object_value:
            most_dissm_object_value = diff
            most_dissm_object_index = ele
    if(most_dissm_object_value>0):
        return  (most_dissm_object_index, 1)
    else:
        return (-1, -1)
    
def split(element_list):
    main_list = element_list
    splinter_group = []    
    (most_dissm_object_index,flag) = splinter(main_list, splinter_group)
    while(flag > 0):
        main_list.remove(most_dissm_object_index)
        splinter_group.append(most_dissm_object_index)
        (most_dissm_object_index,flag) = splinter(element_list, splinter_group)
    
    return (main_list, splinter_group)

def max_diameter(cluster_list):
    max_diameter_cluster_index = None
    max_diameter_cluster_value = -np.inf
    index = 0
    for element_list in cluster_list:
        for i in element_list:
            for j in element_list:
                if dissimilarity_matrix[i][j]  > max_diameter_cluster_value:
                    max_diameter_cluster_value = dissimilarity_matrix[i][j]
                    max_diameter_cluster_index = index
        
        index +=1
    
    if(max_diameter_cluster_value <= 0):
        return -1
    
    return max_diameter_cluster_index

In [61]:
current_clusters = ([labels])
level = 1
index = 0
while(index!=-1):
    print(level, current_clusters)
    (a_clstr, b_clstr) = split(current_clusters[index])
    del current_clusters[index]
    current_clusters.append(a_clstr)
    current_clusters.append(b_clstr)
    index = max_diameter(current_clusters)
    level +=1
    print()
print(level, current_clusters)

1 [['A', 'B', 'C', 'D', 'E']]

2 [['A', 'C', 'D'], ['B', 'E']]

3 [['B', 'E'], ['A', 'C'], ['D']]

4 [['A', 'C'], ['D'], ['E'], ['B']]

5 [['D'], ['E'], ['B'], ['C'], ['A']]
